In [17]:
import json
import time
import datetime
import requests 
import pandas as pd
import numpy as np
import yaml
from pathlib import Path
import pickle

# API Key IlbRy4iSNXBdEQhTyhQLXl68EVS3U2XttiNvLtpW
# 1000 Calls / Hour
# Put API Key in file. Read in API key from file instead of hardcoding in jupyter notebook. 
# http://petstore.swagger.io/
# https://api.open.fec.gov/swagger/

In [4]:
# CONFIG_FILE = Path('.')/Path('0-setup')/Path('fec_config.yaml')
# with open(CONFIG_FILE, 'r') as f:
#     config = yaml.load(f)

In [3]:
api_key = 'IlbRy4iSNXBdEQhTyhQLXl68EVS3U2XttiNvLtpW'

In [9]:
#Retrieve Comprehensive Committee Dictionary

i = 1
committee_dict = []
while True:
    url = 'https://api.open.fec.gov/v1/committees/\
?per_page=100&sort_null_only=false&sort=name\
&sort_hide_null=false\
&page='+str(i) + '\
&min_first_file_date=2010\
&api_key=' + api_key

    committee_id = requests.get(url)
    if committee_id.json()['results'] ==  []:
        break
    else: 
        results = committee_id.json()['results']
        committee_dict = committee_dict +  results
        i += 1
print('Done')
print(i)

df_general_comm = pd.DataFrame(committee_dict)
df_general_comm.head()

df_committee_id = df_general_comm.drop(columns=['organization_type', 'organization_type_full', 'party', 'party_full', 'treasurer_name'])
empty = []
df_committee_id = df_committee_id.loc[df_committee_id.candidate_ids.map(len) > 0].reset_index()
df_committee_id.head()

# Add column for metric to weight the effect of contribution 
# on each candidate represented by committee

# Simple approach, to be updated at a later point

number_candidates = np.array([len(x) for x in df_committee_id.candidate_ids])**float(-1)
df_committee_id['candidate_donor_weight'] = number_candidates

df_committee_id.head()

Done
199


,index,candidate_ids,committee_id,committee_type,committee_type_full,cycles,designation,designation_full,filing_frequency,first_file_date,last_f1_date,last_file_date,state,candidate_donor_weight
0,0,[P00008052],C00684373,P,Presidential,[2018],P,Principal campaign committee,Q,2018-08-03,2018-08-03,2018-09-09,MA,1.000
1,1,[S8MN00438],C00555748,N,PAC - Nonqualified,[2014],J,Joint fundraising committee,T,2014-01-28,2014-01-28,2014-12-18,DC,1.000
2,30,"[S0MA00109, S2NV00183, S4IN00014, S4ME00055, S...",C00495390,S,Senate,[2012],J,Joint fundraising committee,T,2011-04-15,2011-05-10,2011-10-31,VA,0.125
3,34,"[H6OH04082, H8OH05036]",C00528091,H,House,[2012],J,Joint fundraising committee,T,2012-08-29,2012-08-29,2012-11-29,OH,0.500
4,35,[H6NY25125],C00532879,N,PAC - Nonqualified,"[2012, 2014, 2016, 2018]",J,Joint fundraising committee,Q,2012-10-22,2012-10-22,2018-10-25,DC,1.000


In [12]:
df_general_comm.name
df_committee_id = df_general_comm.drop(columns=['organization_type', 'organization_type_full', 'party', 'party_full', 'treasurer_name'])


In [18]:
# df_committee_id = pd.read_pickle('committee_ids.pkl')
df_committee_id.head()
number_candidates = np.array([len(x) for x in df_committee_id.candidate_ids])**float(-1)
df_committee_id['candidate_donor_weight'] = number_candidates

df_committee_id.to_pickle('committee_ids.pkl')


C:\Users\nicho\Anaconda3\envs\data-x\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in power
  This is separate from the ipykernel package so we can avoid doing imports until


In [19]:
df_committee_id.head()

,candidate_ids,committee_id,committee_type,committee_type_full,cycles,designation,designation_full,filing_frequency,first_file_date,last_f1_date,last_file_date,name,state,candidate_donor_weight
0,[P00008052],C00684373,P,Presidential,[2018],P,Principal campaign committee,Q,2018-08-03,2018-08-03,2018-09-09,:,MA,1.000000
1,[S8MN00438],C00555748,N,PAC - Nonqualified,[2014],J,Joint fundraising committee,T,2014-01-28,2014-01-28,2014-12-18,"10,000 LAKES VICTORY 2014",DC,1.000000
2,[],C00563023,O,Super PAC (Independent Expenditure-Only),[2014],U,Unauthorized,T,2014-05-12,2014-05-16,2014-06-16,10 ^ 9+,CA,inf
3,[],C00687368,O,Super PAC (Independent Expenditure-Only),[2018],U,Unauthorized,Q,2018-09-12,2018-09-12,2018-11-02,11/06/18 VICTORY FUND,MS,inf
4,[],C00686774,H,House,[2018],J,Joint fundraising committee,Q,2018-09-05,2018-09-05,2018-10-21,1106 FUND,DC,inf


In [29]:
# Run this, build committee donor extraction algorithm that adds to the other function. 
# Run for all candidates. Try and find the election results first tho.


# def reformat_committee_df(df):
#     df_temp = df.copy()
#     df_comm = pd.DataFrame(columns = df_temp.columns)
#     i = 1
#     for list_cand_id in df_temp.candidate_ids:
#         print(i)
#         for cand_id in list_cand_id:
#             df_temp.iloc[i-1:i,:] = df_temp.iloc[i-1:i,:].drop(columns='candidate_ids')
#             df_temp.iloc[i-1:i,:]['candidate_ids'] = cand_id
#             df_comm = pd.concat([df_comm, df_temp.iloc[i-1:i,:]])
#         i+=1
#     return df_comm


len_reform_df = sum(df_committee_id['candidate_ids'].apply(lambda row: len(row)))
print('Length of new datframe {}:'.format(len_reform_df))

def reformat_by_comm_cand(df):
    df_temp = df.copy()
    
    # Create the DataFrame with 54446 rows and columns -- all NaNs
    cols = df_temp.columns
    df_comm = pd.DataFrame(np.full((len_reform_df, len(cols)), np.nan), columns=cols)
    df_comm['candidate_ids'] = df_comm['cycles'].astype(list)
    df_comm['cycles'] = df_comm['cycles'].astype(list)
    
    i = 0
    j= 0

    for list_candidates in df_temp.candidate_ids:
        for cand_id in list_candidates:
            df_comm.iloc[j,:] = df_temp.iloc[i,:] 
            df_comm.iloc[j, 0] = cand_id
            j+=1

    
        i+=1
    return df_comm

Length of new datframe 10427:


In [32]:
df_comm = reformat_by_comm_cand(df_committee_id)
# df_comm = reformat_committee_df(df_committee_id)

In [33]:
df_comm.to_pickle('committee_id_formatted_v1.pkl')

In [34]:
df_comm.head()

,candidate_ids,committee_id,committee_type,committee_type_full,cycles,designation,designation_full,filing_frequency,first_file_date,last_f1_date,last_file_date,name,state,candidate_donor_weight
0,P00008052,C00684373,P,Presidential,[2018],P,Principal campaign committee,Q,2018-08-03,2018-08-03,2018-09-09,:,MA,1.000
1,S8MN00438,C00555748,N,PAC - Nonqualified,[2014],J,Joint fundraising committee,T,2014-01-28,2014-01-28,2014-12-18,"10,000 LAKES VICTORY 2014",DC,1.000
2,S0MA00109,C00495390,S,Senate,[2012],J,Joint fundraising committee,T,2011-04-15,2011-05-10,2011-10-31,2011 SENATORS CLASSIC COMMITTEE,VA,0.125
3,S2NV00183,C00495390,S,Senate,[2012],J,Joint fundraising committee,T,2011-04-15,2011-05-10,2011-10-31,2011 SENATORS CLASSIC COMMITTEE,VA,0.125
4,S4IN00014,C00495390,S,Senate,[2012],J,Joint fundraising committee,T,2011-04-15,2011-05-10,2011-10-31,2011 SENATORS CLASSIC COMMITTEE,VA,0.125
